# Setup

In [1]:
CASSANDRA_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

CASSANDRA_CLUSTER_NAME = "cassandra-cluster.mavasbel.vpn.itam.mx"
CASSANDRA_TOTAL_NODES = 3

CASSANDRA_NODE_IPS = ["10.15.20.2"] * CASSANDRA_TOTAL_NODES
CASSANDRA_NODE_NAMES = [f"cassandra-node-{i+1}" for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_HOSTNAMES = [
    f"{CASSANDRA_NODE_NAMES[i]}.mavasbel.vpn.itam.mx"
    for i in range(CASSANDRA_TOTAL_NODES)
]
CASSANDRA_NODE_GOSSIP_PORTS = [7000 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_RPC_PORTS = [9040 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_SSL_GOSSIP_PORTS = [7500 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_JMX_PORTS = [7200 + (i + 1) for i in range(0, CASSANDRA_TOTAL_NODES)]

CASSANDRA_CA_CERT_PASSWORD = "cassandra_cluster_ca_cert_passowrd"
CASSANDRA_NODE_CERT_PASSWORD = "cassandra_cluster_cert_passowrd"
CASSANDRA_INIT_USER = "cassandra"
CASSANDRA_INIT_PASSWORD = "cassandra"

CASSANDRA_WORKDIR = "/var/lib/cassandra"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")
CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR = os.path.join(LOCALHOST_WORKDIR, "cluster_certs")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)
os.environ.setdefault("CQLENG_ALLOW_SCHEMA_MANAGEMENT", "True")

'True'

# Session creation

In [3]:
from cassandra.cluster import Cluster
from cassandra.connection import DefaultEndPoint
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

cassandra_nodes = [
    DefaultEndPoint(CASSANDRA_NODE_HOSTNAMES[j], CASSANDRA_NODE_RPC_PORTS[j])
    for j in range(CASSANDRA_TOTAL_NODES)
]
print(
    f"🔗 Connecting to: {[f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes]}"
)
print(
    f"JDBC URL: jdbc:cassandra://{','.join([f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes])}"
)

auth_provider = PlainTextAuthProvider(
    username=CASSANDRA_INIT_USER, password=CASSANDRA_INIT_PASSWORD
)
cluster = Cluster(contact_points=cassandra_nodes, auth_provider=auth_provider)

session = cluster.connect()
session.row_factory = dict_factory
print(f"✅ Connected to cluster: {cluster.metadata.cluster_name}")
print(f"🌐 Nodes found: {len(cluster.metadata.all_hosts())}")

🔗 Connecting to: ['cassandra-node-1.mavasbel.vpn.itam.mx:9041', 'cassandra-node-2.mavasbel.vpn.itam.mx:9042', 'cassandra-node-3.mavasbel.vpn.itam.mx:9043']
JDBC URL: jdbc:cassandra://cassandra-node-1.mavasbel.vpn.itam.mx:9041,cassandra-node-2.mavasbel.vpn.itam.mx:9042,cassandra-node-3.mavasbel.vpn.itam.mx:9043
✅ Connected to cluster: cassandra-cluster.mavasbel.vpn.itam.mx
🌐 Nodes found: 3


In [4]:
import pprint
import pandas as pd
from IPython.display import display

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)
pd.options.display.html.use_mathjax = True
pd.options.display.html.border = 1
pd.options.display.html.table_schema = False
pd.options.display.expand_frame_repr = True

display(
    pd.DataFrame(
        [cassandra_host.__dict__ for cassandra_host in cluster.metadata.all_hosts()]
    )
    .sort_index()
    .transpose()
)

,0,1,2
endpoint,cassandra-node-3.mavasbel.vpn.itam.mx:9043,10.15.20.2:9041,10.15.20.2:9042
conviction_policy,<cassandra.policies.SimpleConvictionPolicy object at 0x000002637637DE50>,<cassandra.policies.SimpleConvictionPolicy object at 0x000002637838D810>,<cassandra.policies.SimpleConvictionPolicy object at 0x000002637838DBA0>
host_id,93a96850-1842-45fc-8543-0cddae606d2d,e113825e-cadb-43a2-bdec-793fd0cd67ee,4f1a33f3-5359-4367-9073-3507a5e80be8
_datacenter,dc1,dc1,dc1
_rack,rack1,rack1,rack1
lock,<unlocked _thread.RLock object owner=0 count=0 at 0x00000263761CC080>,<unlocked _thread.RLock object owner=0 count=0 at 0x00000263760BE3C0>,<unlocked _thread.RLock object owner=0 count=0 at 0x00000263760E4DC0>
is_up,True,True,True
listen_address,172.19.0.4,NaN,NaN
listen_port,7003.0,NaN,NaN
broadcast_address,10.15.20.2,10.15.20.2,10.15.20.2


In [5]:
# from cassandra.cluster import ResultSet

# peers_v2: ResultSet = session.execute("SELECT * FROM system.peers_v2")
# locals: ResultSet = session.execute("SELECT * FROM system.local")

# for peer_v2 in peers_v2:
#     print(" | ".join([f"{col}: {peer_v2[col]}" for col in peers_v2.column_names]))
# print("=" * 100)
# for local in locals:
#     print(" | ".join([f"{col}: {local[col]}" for col in locals.column_names]))
# print("=" * 100)

### Keyspace

In [7]:
keyspace_name = "generic_analytics"
if CASSANDRA_START_FROM_SCRATCH:
    session.execute(f"DROP KEYSPACE IF EXISTS {keyspace_name}")

In [8]:
keyspace_name = "generic_analytics"
session.execute(
    f"""
CREATE KEYSPACE IF NOT EXISTS {keyspace_name} 
WITH replication = {{
    'class': 'NetworkTopologyStrategy', 
    'dc1': {CASSANDRA_TOTAL_NODES}
}}
"""
)
session.set_keyspace(keyspace_name)

# Create Table
# Partition Key: city (distributes data)
# Clustering Column: user_id (sorts data within city)
session.execute(
    """
CREATE TABLE IF NOT EXISTS user_metrics (
    city text,
    user_id uuid,
    username text,
    session_duration int,
    last_access timestamp,
    PRIMARY KEY (city, user_id)
);
"""
)

### Insert

In [9]:
from faker import Faker

fake = Faker()

In [10]:
import uuid
from datetime import datetime
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel
from cassandra.query import BatchStatement, BatchType

# 1. Prepare your statement outside the loop
query = """
INSERT INTO user_metrics (city, user_id, username, session_duration, last_access)
VALUES (?, ?, ?, ?, ?)
"""
prepared = session.prepare(query)
# prepared.consistency_level = ConsistencyLevel.QUORUM
# statement = SimpleStatement(query, consistency_level=ConsistencyLevel.QUORUM)

# 2. Create the Batch object
# LOGGED ensures atomicity but adds disk overhead, UNLOGGED is faster
batch_records = 200
batch = BatchStatement(batch_type=BatchType.LOGGED)
batch.consistency_level = ConsistencyLevel.QUORUM
print(f"Preparing batch of {batch_records} records...")
for _ in range(batch_records):
    batch.add(
        prepared,
        (
            fake.city(),
            uuid.uuid4(),
            fake.user_name(),
            fake.random_int(min=1, max=3600),
            datetime.now(),
        ),
    )

# 3. Execute the entire batch at once
session.execute(batch)
print("Batch successfully committed to the cluster.")

Preparing batch of 200 records...
Batch successfully committed to the cluster.


### Query

In [11]:
from cassandra.cluster import ResultSet
from typing import cast

print(cast(ResultSet, session.execute("SELECT count(*) FROM user_metrics")).one())

{'count': 200}


In [12]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT * FROM user_metrics LIMIT 100")
for row in rows.current_rows:
    print(row)

{'city': 'East Melinda', 'user_id': UUID('9deaa9d3-5368-4b52-b3fd-b2b84a444f70'), 'last_access': datetime.datetime(2026, 1, 11, 21, 5, 2, 862000), 'session_duration': 567, 'username': 'erika48'}
{'city': 'Andreaview', 'user_id': UUID('b99fc01b-3272-4654-af79-9a22d1b21928'), 'last_access': datetime.datetime(2026, 1, 11, 21, 5, 2, 872000), 'session_duration': 179, 'username': 'icollins'}
{'city': 'Floresview', 'user_id': UUID('777026a4-7613-41c9-a5f3-834789f3e66d'), 'last_access': datetime.datetime(2026, 1, 11, 21, 5, 2, 871000), 'session_duration': 785, 'username': 'lcruz'}
{'city': 'East Reginachester', 'user_id': UUID('f5dd900f-802a-4954-92eb-7b2fe2a69d08'), 'last_access': datetime.datetime(2026, 1, 11, 21, 5, 2, 858000), 'session_duration': 3257, 'username': 'larry98'}
{'city': 'South George', 'user_id': UUID('daf59481-8f85-4e50-99e2-3f3b6c4131c6'), 'last_access': datetime.datetime(2026, 1, 11, 21, 5, 2, 866000), 'session_duration': 1334, 'username': 'edwardstimothy'}
{'city': 'West 

In [13]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT DISTINCT city FROM user_metrics", [])
for row in rows.current_rows:
    print(row)

{'city': 'East Melinda'}
{'city': 'Andreaview'}
{'city': 'Floresview'}
{'city': 'East Reginachester'}
{'city': 'South George'}
{'city': 'West Sylvia'}
{'city': 'New Norma'}
{'city': 'Williamshaven'}
{'city': 'Port Joshuaport'}
{'city': 'Mathewborough'}
{'city': 'New Robertfurt'}
{'city': 'East Anthony'}
{'city': 'Ericville'}
{'city': 'Savagemouth'}
{'city': 'Wolfeton'}
{'city': 'Marshchester'}
{'city': 'Powellhaven'}
{'city': 'South Marieland'}
{'city': 'West Michelle'}
{'city': 'Matthewsborough'}
{'city': 'North Hannahmouth'}
{'city': 'West Christy'}
{'city': 'Zacharybury'}
{'city': 'Clintonchester'}
{'city': 'Lake Deannamouth'}
{'city': 'East William'}
{'city': 'Brittneystad'}
{'city': 'West Sherryport'}
{'city': 'Lindaton'}
{'city': 'Port Deanview'}
{'city': 'Masonchester'}
{'city': 'Lake Johnberg'}
{'city': 'Kellybury'}
{'city': 'West Tinafort'}
{'city': 'Lake Pam'}
{'city': 'Mccormickhaven'}
{'city': 'Edwardfort'}
{'city': 'New Debbieshire'}
{'city': 'West Brittanyport'}
{'city': 

### Find nodes storing random data

In [14]:
import random
import pprint

random_token = random.randint(-9223372036854775808, 9223372036854775807)
query = "SELECT * FROM user_metrics WHERE token(city) >= %s LIMIT 1"
row: dict = cast(ResultSet, session.execute(query, [random_token])).one()
print(f"Random user_metric: {pprint.pformat(row)}")

prepared = session.prepare("SELECT * FROM user_metrics WHERE city=? AND user_id=?")
bound = prepared.bind([row['city'], row['user_id']])
routing_key = bound.routing_key
nodes = cluster.metadata.get_replicas(keyspace_name, routing_key)

print(f"Nodes storing '{row['city']}':")
for node in nodes:
    print(f" - Host: {node.address}, Gossip Port: {node.broadcast_port}")

Random user_metric: {'city': 'Rodneyside',
 'last_access': datetime.datetime(2026, 1, 11, 21, 5, 2, 868000),
 'session_duration': 321,
 'user_id': UUID('8ac09fb7-8d52-4e39-b7ce-f1f56fe149f6'),
 'username': 'jsmith'}
Nodes storing 'Rodneyside':
 - Host: 10.15.20.2, Gossip Port: 7002
 - Host: 10.15.20.2, Gossip Port: 7001
 - Host: cassandra-node-3.mavasbel.vpn.itam.mx, Gossip Port: 7003


### ORM-like

In [15]:
from typing import cast
from cassandra.cqlengine import columns
from cassandra.cqlengine.query import ModelQuerySet
from cassandra.cqlengine.models import Model
from cassandra.cqlengine.management import sync_table, create_keyspace_network_topology
from cassandra.cqlengine import connection

# 1. Connect the engine
connection.set_session(session)


# 2. Define your "Generic" Model
class UserMetrics(Model):
    __table_name__ = "user_metrics"

    # FIRST primary_key=True is the Partition Key
    # city = columns.Text(primary_key=True)
    city = columns.Text(primary_key=True, partition_key=True)

    # SECOND primary_key=True is the Clustering Key
    user_id = columns.UUID(primary_key=True, default=uuid.uuid4)

    # Attributes (Data)
    username = columns.Text(index=True)
    session_duration = columns.Integer()
    last_access = columns.DateTime()


# 3. Create Keyspace and Sync Table (Equivalent to CREATE TABLE)
create_keyspace_network_topology(keyspace_name, {"dc1": CASSANDRA_TOTAL_NODES})
sync_table(UserMetrics)

In [16]:
# 4. Use it like an ORM
# Create UserMetrics
new_metric: UserMetrics = UserMetrics.create(
    city=fake.city(),
    username=fake.name(),
    session_duration=120,
    last_access=datetime.now(),
)
new_metric.save()
print(f"Saved user_metris: {new_metric}")


# 5. Query UserMetrics
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering()
)
for user_metric in user_metrics:
    print(f"Query user_metris: {user_metric}")

Saved user_metris: UserMetrics <city=East Thomasmouth, user_id=ea9cdf90-7d11-4daf-b83a-2e4e3c9662eb>
Query user_metris: UserMetrics <city=East Melinda, user_id=9deaa9d3-5368-4b52-b3fd-b2b84a444f70>
Query user_metris: UserMetrics <city=Andreaview, user_id=b99fc01b-3272-4654-af79-9a22d1b21928>
Query user_metris: UserMetrics <city=Floresview, user_id=777026a4-7613-41c9-a5f3-834789f3e66d>
Query user_metris: UserMetrics <city=East Reginachester, user_id=f5dd900f-802a-4954-92eb-7b2fe2a69d08>
Query user_metris: UserMetrics <city=South George, user_id=daf59481-8f85-4e50-99e2-3f3b6c4131c6>
Query user_metris: UserMetrics <city=West Sylvia, user_id=81da3a3f-438b-4d6c-ab67-dfed91935417>
Query user_metris: UserMetrics <city=New Norma, user_id=0f03d281-4ace-474b-9c13-70f84362300d>
Query user_metris: UserMetrics <city=Williamshaven, user_id=65213319-4887-4fa2-b5e2-2ad0dbf10b5c>
Query user_metris: UserMetrics <city=Port Joshuaport, user_id=f8794d8c-1833-4dd4-826f-a34dfb643af1>
Query user_metris: UserM

### Insert/Create

In [17]:
from typing import cast

cast(ModelQuerySet, UserMetrics.ttl(86400)).create(
    city=fake.city(),
    username=fake.user_name(),
    session_duration=fake.random_int(10, 1000),
    last_access=fake.date_time(),
)

UserMetrics(city='New Ellenville', user_id=UUID('43589352-c7ed-4e48-9aa3-a2d773e9a7f4'), username='hullgina', session_duration=512, last_access=datetime.datetime(2001, 8, 27, 5, 51, 37))

In [18]:
from typing import cast
from cassandra.cqlengine.query import BatchQuery
from cassandra.cqlengine.query import ModelQuerySet

with BatchQuery() as b:
    for _ in range(100):
        cast(ModelQuerySet, UserMetrics.batch(b)).create(
            city=fake.city(),
            username=fake.user_name(),
            session_duration=fake.random_int(10, 1000),
            last_access=fake.date_time(),
        )

### Delete

In [19]:
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering()
)
for user_metric in user_metrics:
    user_metric.delete()
    print(f"Delete user_metris: {user_metric}")

Delete user_metris: UserMetrics <city=Lynnshire, user_id=0d3f807c-0e80-49ee-b4f5-7e1003a42eee>
Delete user_metris: UserMetrics <city=East Melinda, user_id=9deaa9d3-5368-4b52-b3fd-b2b84a444f70>
Delete user_metris: UserMetrics <city=Richardside, user_id=d09f462a-f213-4c1e-a796-200443cafea0>
Delete user_metris: UserMetrics <city=Andreaview, user_id=b99fc01b-3272-4654-af79-9a22d1b21928>
Delete user_metris: UserMetrics <city=Kevinfort, user_id=c9217e37-4ae6-46b2-9081-b27fa46a1688>
Delete user_metris: UserMetrics <city=Floresview, user_id=777026a4-7613-41c9-a5f3-834789f3e66d>
Delete user_metris: UserMetrics <city=East Reginachester, user_id=f5dd900f-802a-4954-92eb-7b2fe2a69d08>
Delete user_metris: UserMetrics <city=South George, user_id=daf59481-8f85-4e50-99e2-3f3b6c4131c6>
Delete user_metris: UserMetrics <city=West Sylvia, user_id=81da3a3f-438b-4d6c-ab67-dfed91935417>
Delete user_metris: UserMetrics <city=New Norma, user_id=0f03d281-4ace-474b-9c13-70f84362300d>
Delete user_metris: UserMetri

In [20]:
# from cassandra.cqlengine import connection

# # This isn't a compaction, but it forces RAM data to disk so the background compaction process can see it.
# connection.execute(f"ALTER TABLE {keyspace_name}.user_metrics WITH gc_grace_seconds = 10")